<a href="https://colab.research.google.com/github/herculesc/MachineLearning_Classification_FeaturesCNN/blob/main/Logistic_Regression_vgg16_2022_IC_INPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Biblioteca**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [ ]:
# y_true = y_teste
# y_pred = model.predict(x_teste)
dic_pevision = {}

def prevision(y_true, y_pred, final_treino):  
  precision = precision_score(y_true, y_pred , average="macro")
  f1 = f1_score(y_true, y_pred , average="macro")
  recall = recall_score(y_true, y_pred , average="macro")

  dic_pevision['f1-Score'] = "{:.4f}".format(f1)
  dic_pevision['Precision'] = "{:.4f}".format(precision)
  dic_pevision['Recall'] = "{:.4f}".format(recall)
  dic_pevision['tempo'] = "{:.4f}".format(final_treino)
  #print(f'F1: {f1:.4f} Precision:{precision:.4f} Recall:{recall:.4f} Tempo{final_treino:.4f}')

  return dic_pevision

In [ ]:
def Load_model(Penalty, C_, Solver, Max_iter):
  # inica a contagem do tempo
  inicio_treino = init_time()

  # Treinamento
  model = LogisticRegression(
                            penalty=Penalty,
                            random_state=0,
                            C= C_,
                            solver=Solver,
                            max_iter= Max_iter,
                            multi_class = 'auto').fit( x_treino, y_treino)
  final_treino = time_ex(inicio_treino)

  return {'Model':model, 'Tempo': final_treino}

In [ ]:
import sys
# Abrir local do arquivo

sys.path.append('/content/drive/MyDrive/TensorFlow_IC')

# Carrefar função do arquivo
from Time_execution import init_time
from Time_execution import time_ex
timeres = []

# **Carregar conjunto de dados**

In [ ]:
path = ['VGG_16', 'MobileNet', 'NASNetLarge', 'EfficientNetB0', 'EfficientNetB7', 'EfficientNetV2L' ]
train_path = []
test_path = []
for i in path:
  train = f'/content/drive/MyDrive/TensorFlow_IC/Dados/IC_Extraction_save/{i}/{i}_Train.csv'
  train_path.append(train)
  
  test = f'/content/drive/MyDrive/TensorFlow_IC/Dados/IC_Extraction_save/{i}/{i}_Test.csv'
  test_path.append(test)


In [ ]:
select_cnn = 5
selecao_train = train_path[select_cnn] 
selecao_test = test_path[select_cnn]

print(selecao_train)
print(selecao_test)

/content/drive/MyDrive/TensorFlow_IC/Dados/IC_Extraction_save/EfficientNetV2L/EfficientNetV2L_Train.csv
/content/drive/MyDrive/TensorFlow_IC/Dados/IC_Extraction_save/EfficientNetV2L/EfficientNetV2L_Test.csv


In [ ]:
# Extração do conjunto de treinamento e alvo:
data = pd.read_csv(selecao_train)
alvo = pd.read_csv(selecao_test)

# Remover coluna vazia
data = data.drop(columns=['Unnamed: 0'])
alvo = alvo.drop(columns=['Unnamed: 0'])

In [ ]:
# Separação do conjunto alvo para validação:
esperado, alvo = alvo["Classe"], alvo.drop("Classe", axis=1)

In [ ]:
# Separação do conjunto de treinamento:
subteste = data.drop("Classe", axis=1)
valores = data["Classe"]


# **Preparação do conjunto para Logistic Regression**

In [ ]:
# Criar conjunto de dados:
x_treino, x_teste, y_treino, y_teste = train_test_split(subteste, valores, test_size=0.1)

# **Modelo Regressão Logistica**


In [ ]:
# Parâmetros
Penalty_val	= ['l1','l2']
C_val	= [0.500, 0.750, 1]
Max_inter_val	= [30, 60, 80, 100]

In [ ]:
# Cria dicionario com os dados
lista = []
cont = 0
for i in Penalty_val:
  for y in C_val:
    for z in Max_inter_val:
      valor = [i, y, z]
      lista.append(valor)
      #print(f'Penalty: "{i}"  C_val: {y} Max_val: {z}')

In [ ]:
lista2 = [[ 'l1', 0.500, 30]]

In [ ]:
# Treinamento do modelo
import pandas as pd
cnn_select = path[select_cnn]
print(f'CNN Selecionada: {cnn_select}')
output = pd.DataFrame()
for valores in lista2:
  model_compile = Load_model(valores[0], valores[1], 'saga', valores[2])
  modelo = model_compile['Model']
  tempo = model_compile['Tempo']
  
  y_true = y_teste
  y_pred = modelo.predict(x_teste)
  prev = prevision(y_true, y_pred, tempo)
  df_dictionary = pd.DataFrame([prev])
  output = pd.concat([output, df_dictionary], ignore_index=True)

  print(f'Penalty: {valores[0]}  C_val: {valores[1]} Max_val: {valores[2]} {prev}')

CNN Selecionada: EfficientNetV2L
Penalty: l1  C_val: 0.5 Max_val: 30 {'f1-Score': '0.6925', 'Precision': '0.7778', 'Recall': '0.6818', 'tempo': '2.7183'}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


In [ ]:
# writing to Excel
name_save = f'Logistic_Regression_{cnn_select}_2022.xlsx'
datatoexcel = pd.ExcelWriter(name_save)

  
# write DataFrame to excel
output.to_excel(datatoexcel)
  
# save the excel
datatoexcel.save()

print(f'Arquivo: {name_save} salvo')
print('DataFrame is written to Excel File successfully.')

Arquivo: Logistic_Regression_EfficientNetV2L_2022.xlsx salvo
DataFrame is written to Excel File successfully.
